In [1]:
import cv2
import numpy as np
import os
import glob
from skimage import io, transform

from sklearn import tree
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
path = './data/data/'
classname = [x for x in os.listdir(path) if os.path.isdir(path + x)]

In [3]:
def calcSURF(img):
    surf = cv2.xfeatures2d.SURF_create()
    kp, des = surf.detectAndCompute(img, None)
    return des

In [4]:
def calcFV(features, centers):
    featureVector = np.zeros((1,100))
    for i in range(0, features.shape[0]):
        fi = features[i]
        diffMat = np.tile(fi, (100, 1)) - centers
        sqSum = (diffMat**2).sum(axis = 1)
        dist = sqSum**0.5
        sortedIndices = dist.argsort()
        idx = sortedIndices[0]
        featureVector[0][idx] +=1
    return featureVector

In [5]:
def read_img(path):
    cate = [path + x for x in os.listdir(path) if os.path.isdir(path + x)]    
    for idx, folder in enumerate(cate):
        featureSet = np.float32([]).reshape(0,64)
        for im in glob.glob(folder + '/*.jpg'):
            print('reading the images:%s' % (im))
            img = io.imread(im)
            des = calcSURF(img)
            featureSet = np.append(featureSet, des, axis = 0)
        featureCount = featureSet.shape[0]
        filename = './data/svm/features/'+ classname[idx]+ '.npy'
        np.save(filename, featureSet)

In [6]:
def learnVocabulary():
    wordCount = 100
    for i in range(len(classname)):
        filename = './data/svm/features/'+ classname[i]+ '.npy'
        features = np.load(filename)
        
        print('Learning vocabulary of {}.'.format(classname[i]))
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 8, 1)  
        flags = cv2.KMEANS_RANDOM_CENTERS  
        compactness, labels, centers = cv2.kmeans(data = features, K = wordCount,bestLabels = None, criteria = criteria, attempts = 20, flags = flags)  
        
        filename = "./data/svm/vocabulary/" + classname[i] + ".npy"  
        np.save(filename, (labels, centers))  

In [7]:
def predict_response(response_true,response_predict):
    correct = (response_true==response_predict)
    return correct.mean()

In [8]:
def dataProcess():
    data = np.float32([]).reshape(0,100)
    response = np.float32([])
    
    dicIdx = 0
    for i in range(len(classname)):
        imgpath = './data/data/'+ classname[i]
        labels, centers = np.load("./data/svm/vocabulary/" + classname[i] + ".npy") 
        for im in glob.glob(imgpath + '/*.jpg'):
            print('reading the images:%s' % (im))
            img = io.imread(im)
            features = calcSURF(img)
            featureVector = calcFV(features, centers)
            data = np.append(data, featureVector, axis = 0)
        res = np.repeat(np.float32([dicIdx]), data.shape[0])
        response = np.append(response, res)
        dicIdx += 1
    
    response = response.reshape(-1,1)
    num_example = data.shape[0]
    arr = np.arange(num_example)
    np.random.shuffle(arr)
    data = data[arr]
    response = response[arr]
    response = response.flatten()
    
    ratio = 0.8
    s = np.int(num_example * ratio)
    images_train = data[:s]
    images_test = data[s:]
    response_train = response[:s]
    response_test = response[s:]
    
    return images_train, images_test, np.asarray(response_train, np.int32), np.asarray(response_test, np.int32)

In [9]:
#read_img(path = path)

In [10]:
#learnVocabulary()

In [11]:
images_train, images_test, response_train, response_test = dataProcess()

reading the images:./data/data/scatter plot/63.jpg
reading the images:./data/data/scatter plot/77.jpg
reading the images:./data/data/scatter plot/837.jpg
reading the images:./data/data/scatter plot/638.jpg
reading the images:./data/data/scatter plot/604.jpg
reading the images:./data/data/scatter plot/162.jpg
reading the images:./data/data/scatter plot/176.jpg
reading the images:./data/data/scatter plot/610.jpg
reading the images:./data/data/scatter plot/348.jpg
reading the images:./data/data/scatter plot/412.jpg
reading the images:./data/data/scatter plot/374.jpg
reading the images:./data/data/scatter plot/566.jpg
reading the images:./data/data/scatter plot/200.jpg
reading the images:./data/data/scatter plot/758.jpg
reading the images:./data/data/scatter plot/770.jpg
reading the images:./data/data/scatter plot/764.jpg
reading the images:./data/data/scatter plot/765.jpg
reading the images:./data/data/scatter plot/771.jpg
reading the images:./data/data/scatter plot/201.jpg
reading the im

reading the images:./data/data/scatter plot/510.jpg
reading the images:./data/data/scatter plot/276.jpg
reading the images:./data/data/scatter plot/538.jpg
reading the images:./data/data/scatter plot/289.jpg
reading the images:./data/data/scatter plot/464.jpg
reading the images:./data/data/scatter plot/302.jpg
reading the images:./data/data/scatter plot/672.jpg
reading the images:./data/data/scatter plot/114.jpg
reading the images:./data/data/scatter plot/100.jpg
reading the images:./data/data/scatter plot/128.jpg
reading the images:./data/data/scatter plot/29.jpg
reading the images:./data/data/scatter plot/15.jpg
reading the images:./data/data/scatter plot/841.jpg
reading the images:./data/data/scatter plot/699.jpg
reading the images:./data/data/scatter plot/843.jpg
reading the images:./data/data/scatter plot/658.jpg
reading the images:./data/data/scatter plot/102.jpg
reading the images:./data/data/scatter plot/499.jpg
reading the images:./data/data/scatter plot/328.jpg
reading the im

reading the images:./data/data/scatter plot/320.jpg
reading the images:./data/data/scatter plot/446.jpg
reading the images:./data/data/scatter plot/452.jpg
reading the images:./data/data/scatter plot/334.jpg
reading the images:./data/data/scatter plot/491.jpg
reading the images:./data/data/scatter plot/644.jpg
reading the images:./data/data/scatter plot/122.jpg
reading the images:./data/data/scatter plot/650.jpg
reading the images:./data/data/scatter plot/678.jpg
reading the images:./data/data/scatter plot/687.jpg
reading the images:./data/data/scatter plot/23.jpg
reading the images:./data/data/scatter plot/37.jpg
reading the images:./data/data/scatter plot/697.jpg
reading the images:./data/data/scatter plot/683.jpg
reading the images:./data/data/scatter plot/33.jpg
reading the images:./data/data/scatter plot/27.jpg
reading the images:./data/data/scatter plot/126.jpg
reading the images:./data/data/scatter plot/640.jpg
reading the images:./data/data/scatter plot/668.jpg
reading the imag

reading the images:./data/data/scatter plot/587.jpg
reading the images:./data/data/scatter plot/593.jpg
reading the images:./data/data/scatter plot/578.jpg
reading the images:./data/data/scatter plot/222.jpg
reading the images:./data/data/scatter plot/544.jpg
reading the images:./data/data/scatter plot/236.jpg
reading the images:./data/data/scatter plot/785.jpg
reading the images:./data/data/scatter plot/752.jpg
reading the images:./data/data/scatter plot/753.jpg
reading the images:./data/data/scatter plot/747.jpg
reading the images:./data/data/scatter plot/790.jpg
reading the images:./data/data/scatter plot/784.jpg
reading the images:./data/data/scatter plot/551.jpg
reading the images:./data/data/scatter plot/237.jpg
reading the images:./data/data/scatter plot/223.jpg
reading the images:./data/data/scatter plot/545.jpg
reading the images:./data/data/scatter plot/425.jpg
reading the images:./data/data/scatter plot/343.jpg
reading the images:./data/data/scatter plot/357.jpg
reading the 

reading the images:./data/data/bar chart/613.jpg
reading the images:./data/data/bar chart/161.jpg
reading the images:./data/data/bar chart/388.jpg
reading the images:./data/data/bar chart/439.jpg
reading the images:./data/data/bar chart/377.jpg
reading the images:./data/data/bar chart/411.jpg
reading the images:./data/data/bar chart/405.jpg
reading the images:./data/data/bar chart/559.jpg
reading the images:./data/data/bar chart/203.jpg
reading the images:./data/data/bar chart/565.jpg
reading the images:./data/data/bar chart/571.jpg
reading the images:./data/data/bar chart/217.jpg
reading the images:./data/data/bar chart/798.jpg
reading the images:./data/data/bar chart/773.jpg
reading the images:./data/data/bar chart/772.jpg
reading the images:./data/data/bar chart/766.jpg
reading the images:./data/data/bar chart/799.jpg
reading the images:./data/data/bar chart/570.jpg
reading the images:./data/data/bar chart/216.jpg
reading the images:./data/data/bar chart/202.jpg
reading the images:.

...

reading the images:./data/data/rectangle treemap/52.jpg
reading the images:./data/data/rectangle treemap/0.jpg
reading the images:./data/data/rectangle treemap/806.jpg
reading the images:./data/data/rectangle treemap/46.jpg
reading the images:./data/data/rectangle treemap/184.jpg
reading the images:./data/data/rectangle heatmap/63.jpg
reading the images:./data/data/rectangle heatmap/189.jpg
reading the images:./data/data/rectangle heatmap/77.jpg
reading the images:./data/data/rectangle heatmap/837.jpg
reading the images:./data/data/rectangle heatmap/162.jpg
reading the images:./data/data/rectangle heatmap/88.jpg
reading the images:./data/data/rectangle heatmap/610.jpg
reading the images:./data/data/rectangle heatmap/348.jpg
reading the images:./data/data/rectangle heatmap/360.jpg
reading the images:./data/data/rectangle heatmap/406.jpg
reading the images:./data/data/rectangle heatmap/374.jpg
reading the images:./data/data/rectangle heatmap/228.jpg
reading the images:./data/data/rectang

reading the images:./data/data/rectangle heatmap/628.jpg
reading the images:./data/data/rectangle heatmap/614.jpg
reading the images:./data/data/rectangle heatmap/172.jpg
reading the images:./data/data/rectangle heatmap/166.jpg
reading the images:./data/data/rectangle heatmap/98.jpg
reading the images:./data/data/rectangle heatmap/833.jpg
reading the images:./data/data/rectangle heatmap/73.jpg
reading the images:./data/data/rectangle heatmap/199.jpg
reading the images:./data/data/rectangle heatmap/827.jpg
reading the images:./data/data/rectangle heatmap/67.jpg
reading the images:./data/data/rectangle heatmap/9.jpg
reading the images:./data/data/rectangle heatmap/840.jpg
reading the images:./data/data/rectangle heatmap/14.jpg
reading the images:./data/data/rectangle heatmap/854.jpg
reading the images:./data/data/rectangle heatmap/883.jpg
reading the images:./data/data/rectangle heatmap/129.jpg
reading the images:./data/data/rectangle heatmap/673.jpg
reading the images:./data/data/rectan

reading the images:./data/data/rectangle heatmap/703.jpg
reading the images:./data/data/rectangle heatmap/717.jpg
reading the images:./data/data/rectangle heatmap/529.jpg
reading the images:./data/data/rectangle heatmap/501.jpg
reading the images:./data/data/rectangle heatmap/267.jpg
reading the images:./data/data/rectangle heatmap/449.jpg
reading the images:./data/data/rectangle heatmap/475.jpg
reading the images:./data/data/rectangle heatmap/313.jpg
reading the images:./data/data/rectangle heatmap/307.jpg
reading the images:./data/data/rectangle heatmap/461.jpg
reading the images:./data/data/rectangle heatmap/893.jpg
reading the images:./data/data/rectangle heatmap/887.jpg
reading the images:./data/data/rectangle heatmap/677.jpg
reading the images:./data/data/rectangle heatmap/663.jpg
reading the images:./data/data/rectangle heatmap/850.jpg
reading the images:./data/data/rectangle heatmap/688.jpg
reading the images:./data/data/rectangle heatmap/844.jpg
reading the images:./data/data/

reading the images:./data/data/rectangle heatmap/641.jpg
reading the images:./data/data/rectangle heatmap/899.jpg
reading the images:./data/data/rectangle heatmap/655.jpg
reading the images:./data/data/rectangle heatmap/133.jpg
reading the images:./data/data/rectangle heatmap/866.jpg
reading the images:./data/data/rectangle heatmap/872.jpg
reading the images:./data/data/rectangle heatmap/32.jpg
reading the images:./data/data/rectangle heatmap/682.jpg
reading the images:./data/data/rectangle heatmap/696.jpg
reading the images:./data/data/rectangle heatmap/680.jpg
reading the images:./data/data/rectangle heatmap/18.jpg
reading the images:./data/data/rectangle heatmap/858.jpg
reading the images:./data/data/rectangle heatmap/24.jpg
reading the images:./data/data/rectangle heatmap/870.jpg
reading the images:./data/data/rectangle heatmap/643.jpg
reading the images:./data/data/rectangle heatmap/125.jpg
reading the images:./data/data/rectangle heatmap/657.jpg
reading the images:./data/data/rec

reading the images:./data/data/rectangle heatmap/838.jpg
reading the images:./data/data/rectangle heatmap/186.jpg
reading the images:./data/data/rectangle heatmap/2.jpg
reading the images:./data/data/rectangle heatmap/50.jpg
reading the images:./data/data/rectangle heatmap/810.jpg
reading the images:./data/data/rectangle heatmap/44.jpg
reading the images:./data/data/rectangle heatmap/151.jpg
reading the images:./data/data/rectangle heatmap/145.jpg
reading the images:./data/data/rectangle heatmap/93.jpg
reading the images:./data/data/rectangle heatmap/87.jpg
reading the images:./data/data/rectangle heatmap/179.jpg
reading the images:./data/data/rectangle heatmap/384.jpg
reading the images:./data/data/rectangle heatmap/435.jpg
reading the images:./data/data/rectangle heatmap/353.jpg
reading the images:./data/data/rectangle heatmap/421.jpg
reading the images:./data/data/rectangle heatmap/409.jpg
reading the images:./data/data/rectangle heatmap/582.jpg
reading the images:./data/data/rectan

In [12]:
ESTIMATORS = {
    "Linear regression": LinearRegression(),
    "SVC": SVC(),
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": tree.DecisionTreeClassifier(),
}

In [13]:
images_test_predict = dict()
for name, estimator in ESTIMATORS.items():
    print("estimator: {} fit start".format(name))
    estimator.fit(images_train, response_train)
    print("estimator: {} fit complete".format(name))
    images_test_predict[name] = estimator.predict(images_test)
    print("predict_complete: {}".format(name))

estimator: Linear regression fit start
estimator: Linear regression fit complete
predict_complete: Linear regression
estimator: SVC fit start
estimator: SVC fit complete
predict_complete: SVC
estimator: Random Forest fit start
estimator: Random Forest fit complete
predict_complete: Random Forest
estimator: Decision Tree fit start
estimator: Decision Tree fit complete
predict_complete: Decision Tree


In [14]:
for name, estimator in ESTIMATORS.items():
    predict = predict_response(response_test,images_test_predict[name])
    print("{0}: {1:.1%}".format(name,predict))

Linear regression: 0.0%
SVC: 42.1%
Random Forest: 89.7%
Decision Tree: 86.9%
